In [1]:
import camelot
import pandas as pd
import openpyxl
import os
import pdfplumber
import re
from datetime import datetime
# tables = camelot.read_pdf('epd1388_2021.pdf', pages='10',process_background=True)
# tables = camelot.read_pdf('./IBU-09-20/New/14632.pdf', pages='6', split_text=True,table_areas=["69.60548391889084,500.088593765987,525.3099336281931,403.96614296894364"])
# print(len(tables))

# tables = camelot.read_pdf('./New/14699.pdf', pages='all')
# print(len(tables))

In [47]:
columns = {
    "impactGWP100_kgCO2e_total": '\(kg CO2 eq.\)\nGWPTOTAL',
    "impactGWP100_kgCO2e_fossil": '\(kg CO2 eq.\)\nGWPFOSSIL',
    "impactGWP100_kgCO2e_biogenic": '\(kg CO2 eq.\)\nGWPBIOGENIC',
    "impactGWP100_kgCO2e_luluc": '\(kg CO2 eq.\)\nGWPLULUC',
    "impactODP_WMO2014_kgCFC11e": 'ODP\n\(kg CFC 11 eq.\)',
    "impactAP_molHeq": 'AP\n\(mol H\+ eq\.\)',
    "impactEPFreshwater_kgPO4e": '\(kg \(PO4\)3- eq.\)\nEPFRESHWATER',
    "impactEPFreshwater_kgP": '\(kg P eq.\)\nEPFRESHWATER',
    "impactEPMarine_kgNe": '\(kg N eq.\)\nEPMARINE',
    "impactEPTerrestrial_molNe": '\(mol N eq.\)\nEPTERRESTRIAL',
    "impactPOCP_kgNMVOCe": 'POCP\n\(kg NMVOC eq.\)',
    
    "cleanWaterDeprivation_m3": '^WDP\*\n\(m3 world eq.\)',
    "particulateMatter_incidence": '^PM',
    "ionisingRadiation_kgU235eq": '^IR',
    "ecoToxicityFreshwater_CTUe": '^ETP - FW',
    "humanToxicityCancer_CTUh": '^HTTP - C',
    "humanToxicityNonCancer_CTUh": '^HTTP - NC',
    "potentialSoilQualityIndex": '^SQP',
  "impactGWP100_kgCO2e": "(^GWP - Total$)|(^Global warming potential \xad total$)|(^GWP-total$)|(^GWP$)|(^Global warming potential$)|(^Globales Erwärmungspotenzial ­ total$)|(^Globales Erwärmungspotenzial$)",
  "impactODP_kgCFC11e": "(^ODP)|((^Abbaupotenzial der stratosphärischen Ozonschicht$)|(^Depletion potential of the stratospheric ozone layer$)",
  "impactAP_kgSO2e": "(^AP)|(^AP Air$)|(^Versauerungspotenzial von Boden und Wasser$)|(^Acidification potential of land and water$)",
  "impactEP_kgPO4e": "(^Eutrophication, fraction of nutrients reaching freshwater \nend compartment$)|(^EP-freshwater$)|(^EP$)|(^Eutrophierungspotenzial$)|(^Eutrophication potential)",
  "impactPOCP_kgEthenee": "(^POCP$)|(^Bildungspotenzial für troposphärisches Ozon$)|(^Formation potential of tropospheric ozone photochemical \noxidants)",
  "impactADPElements_kgSbe": "(^\(kg Sb eq.\)\nADPMINERALS&METALS\*)|(^Abiotic depletion potential for non\xadfossil resources$)|(^ADPE$)|(^Potenzial für die Verknappung abiotischer Ressourcen \xad \nnicht fossile Ressourcen$)|(^Potenzial für die Verknappung abiotischer Ressourcen - \nnicht fossile Ressourcen$)|(^Abiotic depletion potential for non-fossil resources)",
  "impactADPFossilFuels_MJ": "(^\(MJ\)\nADPFOSSIL\*)|(^ADPF)|(^Potenzial für die Verknappung abiotischer Ressourcen \xad \nfossile Brennstoffe$)|(^Potenzial für die Verknappung abiotischer Ressourcen - \nfossile Brennstoffe$)|(^Abiotic depletion potential for fossil resources)",
    
  "renewablesUsedAsEnergy_MJ": "(^PERE)|(^Erneuerbare Primärenergie als Energieträger$)|(^Renewable primary energy as energy carrier$)",
  "renewablesUsedAsMaterial_MJ": "(^PERM)|(^Erneuerbare Primärenergie zur stofflichen Nutzung$)|(^Renewable primary energy resources as material utilization$)",
  "embodiedRenewableEnergy_MJ": "(^PERT)|(^Total erneuerbare Primärenergie$)|(^Total use of renewable primary energy resources$)",
  "nonRenewablesUsedAsEnergy_MJ": "(^PENRE)|(^Non\xadrenewable primary energy as energy carrier$)|(^Nicht-erneuerbare Primärenergie als Energieträger$)|(^Nicht\xaderneuerbare Primärenergie als Energieträger$)|(^Non-renewable primary energy as energy carrier$)",
  "nonRenewablesUsedAsMaterial_MJ": "(^PENRM)|(^Non\xadrenewable primary energy as material utilization$)|(^Nicht-erneuerbare Primärenergie zur stofflichen Nutzung$)|(^Nicht\xaderneuerbare Primärenergie zur stofflichen Nutzung$)|(^Non-renewable primary energy as material utilization$)",
  "embodiedEnergy_MJ": "(^PENRT)|(^Total nicht erneuerbare Primärenergie$)|(^Total use of non-renewable primary energy resources$)|(^Total use of non\xadrenewable primary energy resources$)",
  "recyclingMaterialUse_kg": "(^SM)|(^Einsatz von Sekundärstoffen$)|(^Use of secondary material$)",
  "renewableRecylingFuelUse_MJ": "(^RSF)|(^Erneuerbare Sekundärbrennstoffe$)|(^Use of renewable secondary fuels$)",
  "nonRenewableRecylingFuelUse_MJ": "(^NRSF)|(^Use of non\xadrenewable secondary fuels$)|(^Nicht-erneuerbare Sekundärbrennstoffe$)|(^Nicht\xaderneuerbare Sekundärbrennstoffe$)|(^Use of non-renewable secondary fuels$)",
  "cleanWaterNetUse_m3": "(^FW)|(^Nettoeinsatz von Süßwasserressourcen$)|(^Use of net fresh water$)",
    
  "wasteHazardous_kg": "(^HWD)|(^Gefährlicher Abfall zur Deponie$)|(^Hazardous waste disposed$)",
  "wasteNonHazardous_kg": "(^NHWD)|(^Non\xadhazardous waste disposed$)|(^Entsorgter nicht gefährlicher Abfall$)|(^Non-hazardous waste disposed$)",
  "wasteRadioactive_kg": "(^RWD)|(^Entsorgter radioaktiver Abfall$)|(^Radioactive waste disposed$)",
  "reusableMaterialsOutput_kg": "(^CRU)|(^Komponenten für die Wiederverwendung$)|(^Components for re-use$)|(^Components for re\xaduse$)",
  "recyclableMaterialsOutput_kg": "(^MFR)|(^Stoffe zum Recycling$)|(^Materials for recycling$)",
  "energyMaterialsOutput_kg": "(^MER)|(^Stoffe für die Energierückgewinnung)|(^Materials for energy recovery$)",
  "exportedEnergyOutput_MJ_electric": "(^EEE)|(^Exportierte elektrische Energie$)|(^Exported electrical energy$)",
  "exportedEnergyOutput_MJ_thermal": "(^EET)|(^Exportierte thermische Energie$)|(^Exported thermal energy$)",
  "exportedEnergyOutput_MJ": "(^EE)",
    
  "traciGWP_kgCO2e": 8,
  "traciODP_kgCFC11e": 12,
  "traciAP_kgSO2e": 6,
  "traciEP_kgNe": 7,
  "traciPOCP_kgO3e": 10,
  "traciNRPE_MJ": 17
}

headers = [
    "impactGWP100_kgCO2e_total",
    "impactGWP100_kgCO2e_fossil",
    "impactGWP100_kgCO2e_biogenic",    
    'impactGWP100_kgCO2e_luluc',
    'impactODP_WMO2014_kgCFC11e',
    'impactAP_molHeq',
    'impactEPFreshwater_kgPO4e',
    'impactEPFreshwater_kgP',
    'impactEPMarine_kgNe',
    'impactEPTerrestrial_molNe',
    'impactPOCP_kgNMVOCe',
    'cleanWaterDeprivation_m3',
    'particulateMatter_incidence',
    'ionisingRadiation_kgU235eq',
    'ecoToxicityFreshwater_CTUe',
    'humanToxicityCancer_CTUh',
    'humanToxicityNonCancer_CTUh',
    'potentialSoilQualityIndex',
    
    "impactODP_kgCFC11e",
    "impactAP_kgSO2e",
    "impactEP_kgPO4e",
    "impactPOCP_kgEthenee",
    "impactADPElements_kgSbe",
    "impactADPFossilFuels_MJ",
    
    "renewablesUsedAsEnergy_MJ",
    "renewablesUsedAsMaterial_MJ",
    "embodiedRenewableEnergy_MJ",
    "nonRenewablesUsedAsEnergy_MJ",
    "nonRenewablesUsedAsMaterial_MJ",
    "embodiedEnergy_MJ",
    
    "recyclingMaterialUse_kg",
    "renewableRecylingFuelUse_MJ",
    "nonRenewableRecylingFuelUse_MJ",
    "cleanWaterNetUse_m3",
    "wasteHazardous_kg",
    "wasteNonHazardous_kg",
    "wasteRadioactive_kg",
    "reusableMaterialsOutput_kg",
    "recyclableMaterialsOutput_kg",
    "energyMaterialsOutput_kg",
    "exportedEnergyOutput_MJ_thermal",
    "exportedEnergyOutput_MJ_electric",
    "exportedEnergyOutput_MJ",
    
    "traciGWP_kgCO2e",
    "traciODP_kgCFC11e",
    "traciAP_kgSO2e",
    "traciEP_kgNe",
    "traciPOCP_kgO3e",
    "traciNRPE_MJ"
]

In [48]:
def epd_number(file_name):
    with pdfplumber.open('./Group 1/' + file_name) as pdf:
        first_page = pdf.pages[0]
        text = first_page.extract_text()
        #print(text)
        #item = re.findall('EPD registration number.*', text)
        #try:
        #    r = item[0].split(' ')[3]
        #except:
        #    r = '' 
        item = re.findall('S-P-.*', text)
        
    return item[0]

In [49]:

def make_excel_file(files_data):
    book = openpyxl.Workbook()
    filename = 'Envirodec_' + datetime.now().strftime("%Y-%m-%d") + '.xlsx'
    book.save(filename)

    sheet = book['Sheet']
    sheet.cell(row=1, column=1).value = 'Filename'
    sheet.cell(row=1, column=2).value = 'EPD_Number'
    sheet.cell(row=1, column=3).value = 'DownloadLink'
    sheet.cell(row=1, column=4).value = 'Stages'
    
    count = 5
    for header in headers:
        sheet.cell(row=1, column=count).value = header
        count += 1
    
    current_row = 2
    for file_data in files_data:
        print(file_data["file_name"])
        print(file_data["stages"])
        print("*******************")
        df = file_data["dataFrame"]
        base_added = False
        
        for stage in file_data["stages"]:
            if not base_added:
                sheet.cell(row=current_row, column=1).value = file_data["file_name"]
                sheet.cell(row=current_row, column=2).value = epd_number(file_data["file_name"])
                sheet.cell(row=current_row, column=3).value = '/mnt/data/Downloads/2021-12-13/Envirodec/New/' + file_data["file_name"]
                base_added = True

            sheet.cell(row=current_row, column=4).value = stage
            header_counter = 0
            for header in headers:
                
#                 i = file_data["stages"].index(stage) + 2
#                 sheet.cell(row=current_row, column= 5 + header_counter).number_format = '0.000E+00'
#                 value = float(df[df[0].str.contains(columns2[header])][i].to_list()[0])
#                 #value = float(df[df.eq(columns[header]).any(1)][i].to_list()[0])
# #                 sheet.cell(row=current_row, column= 5 + header_counter).value = value
                try:
                    num_of_columns = len(df.columns)                    
                    i = file_data["stages"].index(stage) + (num_of_columns - len(file_data["stages"]))
                    sheet.cell(row=current_row, column= 5 + header_counter).number_format = '0.000E+00'
                    replaced = df[df[0].str.contains(columns[header])][i].to_list()[0].replace(',', '.')
                    if not replaced == 'IND':
                        try:
                            value = float(replaced)
                        except:
                            try:
                                value = float(replaced.replace('\xad', '-'))
                            except:
                                value = float(replaced.replace('\n', ''))
                    else:
                        value = replaced
                        
                    #value = float(df[df.eq(columns[header]).any(1)][i].to_list()[0])
                    sheet.cell(row=current_row, column= 5 + header_counter).value = value
                except:
                    print("**************************************ERRROR*****************************")
                    print(file_data["file_name"])
                    print(stage)
                    print(columns[header])
                header_counter= header_counter +1
            current_row = current_row + 1
        current_row = current_row + 1
            
    book.save(filename)
    book.close()

In [50]:
def clean_df(df):
    transformed = df.applymap(lambda x: x.split(' ') if ('E-' in x) or ('E+' in x) or (x == '') else x)
    for i in range(transformed.shape[0]):
        row = transformed.iloc[[i]]
        for index, element in row.iloc[0].items():
            if element == ['']:
                for index_s, search in row.iloc[0].items():
                    if (len(search) > 1) and ('[kg' not in search) and (type(search) == list):
                        if index < index_s:        
                            transformed.loc[i,index] = [search[0]]
                            row.iloc[0][index] = [search[0]]
                            del search[0]
                        else: 
                            transformed.loc[i,index] = [search[1]]                            
                            row.iloc[0][index] = [search[1]]
                            del search[1]
                        break
    return transformed.applymap(lambda x: x[0] if type(x) == list else x)



In [51]:
files = os.listdir('./Group 1')
# files = ['15230.pdf', '14983.pdf.pdf', '14956.pdf']
r = []
for file in files:
    print("************ FILE NAME: " + file + " ****************")
    tables = camelot.read_pdf('./Group 1/' + file, pages='all')
    dataFrames = []

    file_info = {}
    stages = []
    for table in tables:
        df = table.df
        stages_added = True
        #stages = ['A1', 'A2', 'A3', 'C1', 'C2', 'C3', 'C4', 'Total', 'D']
        #stages = ['A1-A3', 'A4', 'A1-A4']
        #stages = ['A1-A2', 'A3', 'A4', 'C1', 'C2', 'C3', 'C4', 'D']
        stages = ['A1-A3', 'A4', 'A5', 'C1', 'C2', 'C3', 'C4', 'D']
        gwp = ['GWP - Total', '(kg CO2 eq.)\nGWPTOTAL']
        for g in gwp:
            if (g in df.values) and ('TRACI 2.1 Impact Assessment' not in df.values):
                if not stages_added:
                    try:

                        a = list(df[df.eq('Unit').any(1)].values[0])
                        unit_index = a.index('Unit') + 1
                        stages = a[unit_index:]
                    except:
                        a = list(df[df.eq('Einheit').any(1)].values[0])
                        unit_index = a.index('Einheit') + 1
                        stages = a[unit_index:]
                    stages_added = True
                #cleaned_df = clean_df(df)
                #dataFrames.append(cleaned_df)
                print(1)
                dataFrames.append(df)


        pere = ['PERE\nMJ' ]
        for p in pere:
            if p in df.values:
                #cleaned_df = clean_df(df)
                #dataFrames.append(cleaned_df)
                dataFrames.append(df)
                print(2)

        hwd = ['HWD\nkg']
        for h in hwd:
            if h in df.values:
                #cleaned_df = clean_df(df)
                #dataFrames.append(cleaned_df)
                dataFrames.append(df)
                print(3)

                
    result = pd.concat(dataFrames)

    file_info["dataFrame"] = result

        
    file_info["file_name"] = file
    file_info["stages"] = stages
    
    r.append(file_info)
    

make_excel_file(r)


#r

************ FILE NAME: epd1576.pdf ****************
1
2
3
1
2
3
************ FILE NAME: epd2139.pdf ****************
1
2
3
1
2
3
1
2
3
1
2
3
************ FILE NAME: epd2140.pdf ****************
1
2
3
1
2
3
epd1576.pdf
['A1-A3', 'A4', 'A5', 'C1', 'C2', 'C3', 'C4', 'D']
*******************
**************************************ERRROR*****************************
epd1576.pdf
A1-A3
^PM
**************************************ERRROR*****************************
epd1576.pdf
A1-A3
^IR
**************************************ERRROR*****************************
epd1576.pdf
A1-A3
^ETP - FW
**************************************ERRROR*****************************
epd1576.pdf
A1-A3
^HTTP - C
**************************************ERRROR*****************************
epd1576.pdf
A1-A3
^HTTP - NC
**************************************ERRROR*****************************
epd1576.pdf
A1-A3
^SQP
**************************************ERRROR*****************************
epd1576.pdf
A1-A3
(^ODP)|((^Abbaupotenzi

C:\Users\mirol\anaconda3\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


**************************************ERRROR*****************************
epd1576.pdf
D
^PM
**************************************ERRROR*****************************
epd1576.pdf
D
^IR
**************************************ERRROR*****************************
epd1576.pdf
D
^ETP - FW
**************************************ERRROR*****************************
epd1576.pdf
D
^HTTP - C
**************************************ERRROR*****************************
epd1576.pdf
D
^HTTP - NC
**************************************ERRROR*****************************
epd1576.pdf
D
^SQP
**************************************ERRROR*****************************
epd1576.pdf
D
(^ODP)|((^Abbaupotenzial der stratosphärischen Ozonschicht$)|(^Depletion potential of the stratospheric ozone layer$)
**************************************ERRROR*****************************
epd1576.pdf
D
(^Eutrophication, fraction of nutrients reaching freshwater 
end compartment$)|(^EP-freshwater$)|(^EP$)|(^Eutrophierungspotenzial$)|(^

**************************************ERRROR*****************************
epd2139.pdf
C4
(^EEE)|(^Exportierte elektrische Energie$)|(^Exported electrical energy$)
**************************************ERRROR*****************************
epd2139.pdf
C4
(^EE)
**************************************ERRROR*****************************
epd2139.pdf
C4
8
**************************************ERRROR*****************************
epd2139.pdf
C4
12
**************************************ERRROR*****************************
epd2139.pdf
C4
6
**************************************ERRROR*****************************
epd2139.pdf
C4
7
**************************************ERRROR*****************************
epd2139.pdf
C4
10
**************************************ERRROR*****************************
epd2139.pdf
C4
17
**************************************ERRROR*****************************
epd2139.pdf
D
^PM
**************************************ERRROR*****************************
epd2139.pdf
D
^IR
*********

**************************************ERRROR*****************************
epd2140.pdf
D
(^Eutrophication, fraction of nutrients reaching freshwater 
end compartment$)|(^EP-freshwater$)|(^EP$)|(^Eutrophierungspotenzial$)|(^Eutrophication potential)
**************************************ERRROR*****************************
epd2140.pdf
D
(^POCP$)|(^Bildungspotenzial für troposphärisches Ozon$)|(^Formation potential of tropospheric ozone photochemical 
oxidants)
**************************************ERRROR*****************************
epd2140.pdf
D
(^CRU)|(^Komponenten für die Wiederverwendung$)|(^Components for re-use$)|(^Components for re­use$)
**************************************ERRROR*****************************
epd2140.pdf
D
(^MFR)|(^Stoffe zum Recycling$)|(^Materials for recycling$)
**************************************ERRROR*****************************
epd2140.pdf
D
(^MER)|(^Stoffe für die Energierückgewinnung)|(^Materials for energy recovery$)
**********************************

In [3]:
tables = camelot.read_pdf('./Group 1/epd1576.pdf', pages='all')
print(len(tables))

17


In [26]:
tables[4].df

,0,1,2,3,4,5,6,7,8
0,HWD\nkg,"4,70E-03","1,18E-10","5,69E-11","0,00E+00","2,20E-11","0,00E+00","5,55E-11","0,00E+00"
1,NHWD\nkg,"1,54E-02","3,65E-04","7,06E-02","0,00E+00","6,56E-05","0,00E+00","2,61E+00","0,00E+00"
2,RWD\nkg,"7,18E-04","3,04E-06","2,24E-05","0,00E+00","7,58E-07","0,00E+00","5,49E-06","0,00E+00"
3,Components for re-use\nkg,"0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00"
4,Materials for recycling\nkg,"2,15E-01","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00"
5,Materials for energy recovery\nkg,"0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00"
6,"Exported energy, electricity\nMJ","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00"
7,"Exported energy, thermal\nMJ","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00"


In [10]:
'(kg CO2 eq.)\nGWPTOTAL' in tables[4].df.values

True

In [31]:
df = r[0]['dataFrame']

In [32]:

df[df[0].str.contains(columns['impactAP_kgSO2e'])][4].to_list()[0].replace(',', '.')

'0.00E+00'

In [33]:
df[df[0].str.contains(columns['impactGWP100_kgCO2e_total'])][4].to_list()[0].replace(',', '.')

'0.00E+00'

In [34]:
df[df[0].str.contains('^\(kg CO2 eq.\)\nGWPTOTAL')]

,0,1,2,3,4,5,6,7,8
0,(kg CO2 eq.)\nGWPTOTAL,"3,74E+00","2,10E-01","1,95E-01","0,00E+00","3,45E-02","0,00E+00","4,24E-02","0,00E+00"
0,(kg CO2 eq.)\nGWPTOTAL,"3,56E+00","2,02E-01","1,95E-01","0,00E+00","3,31E-02","0,00E+00","4,07E-02","0,00E+00"


In [35]:
df

,0,1,2,3,4,5,6,7,8
0,(kg CO2 eq.)\nGWPTOTAL,"3,74E+00","2,10E-01","1,95E-01","0,00E+00","3,45E-02","0,00E+00","4,24E-02","0,00E+00"
1,(kg CO2 eq.)\nGWPFOSSIL,"3,76E+00","2,09E-01","1,57E-02","0,00E+00","3,43E-02","0,00E+00","4,35E-02","0,00E+00"
2,(kg CO2 eq.)\nGWPBIOGENIC,"-2,34E-02","-2,16E-04","1,79E-01","0,00E+00","-4,12E-05","0,00E+00","-1,26E-03","0,00E+00"
3,(kg CO2 eq.)\nGWPLULUC,"2,59E-03","1,56E-03","1,73E-05","0,00E+00","2,82E-04","0,00E+00","1,28E-04","0,00E+00"
4,ODP\n(kg CFC 11 eq.),"1,42E-07","2,64E-17","2,07E-16","0,00E+00","6,82E-18","0,00E+00","1,69E-16","0,00E+00"
...,...,...,...,...,...,...,...,...,...
3,Components for re-use\nkg,"0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00"
4,Materials for recycling\nkg,"2,28E-01","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00"
5,Materials for energy recovery\nkg,"0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00"
6,"Exported energy, electricity\nMJ","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00","0,00E+00"


In [14]:
'(kg CO2 eq.)\nGWPTOTAL' in df.values

True

In [36]:
columns['impactAP_molHeq'] in df.values

False

In [37]:
'AP\n(mol H+ eq.)' in df.values

True

In [45]:
df[df[0].str.contains('PM.*')]

,0,1,2,3,4,5,6,7,8
8,(kg N eq.)\nEPMARINE,"1,99E-03","7,27E-04","2,40E-05","0,00E+00","9,07E-05","0,00E+00","8,05E-05","0,00E+00"
11,(kg Sb eq.)\nADPMINERALS&METALS*,"1,32E-06","1,51E-08","2,83E-09","0,00E+00","3,06E-09","0,00E+00","4,11E-09","0,00E+00"
8,(kg N eq.)\nEPMARINE,"1,86E-03","6,99E-04","2,41E-05","0,00E+00","8,71E-05","0,00E+00","7,72E-05","0,00E+00"
11,(kg Sb eq.)\nADPMINERALS&METALS*,"1,29E-06","1,45E-08","2,83E-09","0,00E+00","2,94E-09","0,00E+00","3,94E-09","0,00E+00"


In [18]:
tables = camelot.read_pdf('./Group 4/epd4984.pdf', pages='13', process_background=True)
print(len(tables))

2


In [21]:
tables[1].df

,0
0,(cid:31) ‰(cid:11)\nRESOURCE USE\nŠ\n(cid:26)\...
1,(cid:4)\nŸ\n(cid:11)(cid:10)‰(cid:15)”(cid:12)...
2,(cid:12)\n(cid:11)(cid:12)(cid:12)(cid:15)”(ci...
3,Ÿ\n(cid:11)(cid:10)‰(cid:15)”(cid:12)‰\n(cid:1...
4,ž (cid:4)ž (cid:4)ž (cid:4)ž (cid:4)ž (cid:4)ž...
5,(cid:12)\n(cid:11)(cid:12)(cid:12)(cid:15)”(ci...
6,(cid:12) (cid:12) (cid:12) (cid:12) (cid:12) (...
7,‚(cid:27)\n‰\n(cid:11)(cid:10)(cid:6)(cid:15)”...
8,(cid:4)\n(cid:7)\n(cid:11)(cid:12)(cid:9)(cid:...
9,ž (cid:4)ž (cid:1)(cid:9)\nŽ\n(cid:11)•(cid:12...
